In [81]:
import pandas as pd
import numpy as np
import evaluate

import os
import gc

from tqdm.notebook import tqdm
import PIL

import torch
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer, AutoImageProcessor
from datasets import Dataset, Image
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [82]:
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as tt

from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.models import ResNet18_Weights
from tqdm.notebook import tqdm
from torchvision import models as vision_models
import timm




In [83]:
from sklearn.preprocessing import LabelEncoder
data = pd.read_csv(r"D:\data science\T1_hol\dataset\annotations\train_labels.csv")
le1 = LabelEncoder()
data['image_name'] = data['image_id']
data['unified_class'] = data['class']
data['class_id'] = le1.fit_transform(data['class'])
data['class_id'] = data['class_id'].astype('int64')
images_path = r"D:\data science\T1_hol\dataset\train"

# Разобъем данные на тренировочную и отложенную (на которой мы будем проверять качество работы) части
train, val = train_test_split(data, test_size=0.1, random_state=1, stratify=data['class'])

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

# Получилось примерно 28.1к картинок для тренировки и 12.0к картинок для подсчета метрик
print(train.shape, val.shape)

(2160, 5) (240, 5)


In [84]:
data

,image_id,class,image_name,unified_class,class_id
0,image_80487.jpg,rose,image_80487.jpg,rose,2
1,image_32698.jpg,rose,image_32698.jpg,rose,2
2,image_50287.jpg,rose,image_50287.jpg,rose,2
3,image_40321.jpg,rose,image_40321.jpg,rose,2
4,image_98507.jpg,rose,image_98507.jpg,rose,2
...,...,...,...,...,...
2395,image_62880.jpg,fern,image_62880.jpg,fern,1
2396,image_57775.jpg,fern,image_57775.jpg,fern,1
2397,image_41991.jpg,fern,image_41991.jpg,fern,1
2398,image_13204.jpg,fern,image_13204.jpg,fern,1


In [85]:
val = val.drop(['class','image_id'],axis = 1)

In [86]:
train = train.drop(['class','image_id'],axis = 1)

In [87]:
data = data.drop(['class','image_id'],axis = 1)

In [88]:
data

,image_name,unified_class,class_id
0,image_80487.jpg,rose,2
1,image_32698.jpg,rose,2
2,image_50287.jpg,rose,2
3,image_40321.jpg,rose,2
4,image_98507.jpg,rose,2
...,...,...,...
2395,image_62880.jpg,fern,1
2396,image_57775.jpg,fern,1
2397,image_41991.jpg,fern,1
2398,image_13204.jpg,fern,1


In [89]:
data['image_name'] = r'D:/data science/T1_hol/dataset/train/'+data['image_name']

In [90]:
data = data.drop('unified_class',axis = 1)

In [91]:
data = data[data['image_name'].apply(lambda x: len(np.array(PIL.Image.open(x)).shape)==3)]

In [92]:
data

,image_name,class_id
0,D:/data science/T1_hol/dataset/train/image_804...,2
1,D:/data science/T1_hol/dataset/train/image_326...,2
2,D:/data science/T1_hol/dataset/train/image_502...,2
3,D:/data science/T1_hol/dataset/train/image_403...,2
4,D:/data science/T1_hol/dataset/train/image_985...,2
...,...,...
2395,D:/data science/T1_hol/dataset/train/image_628...,1
2396,D:/data science/T1_hol/dataset/train/image_577...,1
2397,D:/data science/T1_hol/dataset/train/image_419...,1
2398,D:/data science/T1_hol/dataset/train/image_132...,1


In [93]:
df = data.reset_index(drop=True)

In [94]:
df

,image_name,class_id
0,D:/data science/T1_hol/dataset/train/image_804...,2
1,D:/data science/T1_hol/dataset/train/image_326...,2
2,D:/data science/T1_hol/dataset/train/image_502...,2
3,D:/data science/T1_hol/dataset/train/image_403...,2
4,D:/data science/T1_hol/dataset/train/image_985...,2
...,...,...
2395,D:/data science/T1_hol/dataset/train/image_628...,1
2396,D:/data science/T1_hol/dataset/train/image_577...,1
2397,D:/data science/T1_hol/dataset/train/image_419...,1
2398,D:/data science/T1_hol/dataset/train/image_132...,1


In [95]:
from datasets import Dataset
import pandas as pd
df = Dataset.from_pandas(df)

In [96]:
df

Dataset({
    features: ['image_name', 'class_id'],
    num_rows: 2400
})

In [97]:
from datasets import Image, Dataset


dataset = df.cast_column("image_name", Image())

In [98]:
dataset = dataset.class_encode_column("class_id")

Stringifying the column:   0%|          | 0/2400 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [99]:
dataset

Dataset({
    features: ['image_name', 'class_id'],
    num_rows: 2400
})

In [100]:
m = "microsoft/swin-base-patch4-window7-224-in22k"

In [101]:
image_processor = AutoImageProcessor.from_pretrained(m)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

d:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danie\.cache\huggingface\hub\models--microsoft--swin-base-patch4-window7-224-in22k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [102]:
splits = dataset.train_test_split(test_size=0.2,stratify_by_column='class_id')
train = splits['train']
val = splits['test']

In [103]:

import torch
from torchvision import transforms

def load_image(example_batch, target_size=(224, 224)):
    """
    Loads and preprocesses images from a batch.

    Args:
        example_batch (dict): A batch of examples.
        target_size (tuple): The desired output size (height, width) of the images.

    Returns:
        dict: The processed batch with pixel values and labels.
    """

    # Define the transformation pipeline
    transform = transforms.Compose([
        transforms.Resize(target_size),  # Resize to the target size
        transforms.ToTensor(),            # Convert to a PyTorch tensor
        transforms.Normalize(          # Normalize pixel values (adjust mean and std as needed)
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Process each image individually and collect pixel values as tensors
    di = []
    for image in example_batch["image_name"]:
        # Ensure image is a PIL Image object
        if isinstance(image, str):
            image = Image.open(image)

        # Convert to RGB if necessary
        if image.mode != "RGB":
            image = image.convert("RGB")
            
        # Apply transformations
        processed_image = transform(image)
        di.append(processed_image)

    # Stack the list of tensors into a single tensor with batch dimension
    example_batch["pixel_values"] = torch.stack(di)
    example_batch["labels"] = torch.tensor(example_batch["class_id"])

    # Remove original columns no longer needed
    del example_batch["image_name"]
    del example_batch["class_id"]

    return example_batch

In [104]:

train.set_transform(load_image)
val.set_transform(load_image)

In [105]:

train[0]

{'pixel_values': tensor([[[ 0.4337,  0.4679,  0.5707,  ...,  0.9646,  1.1872,  1.1187],
          [ 0.5536,  0.7077,  0.7591,  ...,  0.9817,  1.0331,  0.9646],
          [ 0.3652,  0.4679,  0.5193,  ...,  0.9988,  1.1187,  1.0844],
          ...,
          [-1.1932, -1.1760, -1.1418,  ..., -0.4397, -0.4397, -0.4397],
          [-1.2103, -1.1760, -1.1589,  ..., -0.4911, -0.4911, -0.4911],
          [-1.2274, -1.2103, -1.1760,  ..., -0.5082, -0.5082, -0.5082]],
 
         [[-0.7752, -0.7577, -0.6527,  ..., -0.2675, -0.0399, -0.1099],
          [-0.6527, -0.5126, -0.4601,  ..., -0.2500, -0.1975, -0.2675],
          [-0.8452, -0.7402, -0.7052,  ..., -0.2500, -0.1275, -0.1625],
          ...,
          [-2.0007, -1.9832, -1.9832,  ..., -1.9657, -1.9657, -1.9657],
          [-1.9832, -1.9832, -1.9657,  ..., -1.9832, -1.9832, -1.9832],
          [-2.0007, -2.0007, -1.9832,  ..., -1.9832, -1.9832, -1.9832]],
 
         [[-1.3339, -1.3339, -1.2816,  ..., -1.1944, -0.9678, -1.0376],
          [-

In [106]:
metric = evaluate.load("f1")

In [107]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average = 'macro' )

In [108]:
model = AutoModelForImageClassification.from_pretrained(
    m,
    num_labels=10,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/437M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([10, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [109]:
args = TrainingArguments(
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=64,
    #gradient_accumulation_steps=4,
    per_device_eval_batch_size=64,
    num_train_epochs=15,
    #warmup_ratio=0.1,
    #logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    output_dir = 'save',
    # lr_scheduler_type="linear",
)

d:\anaconda\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [110]:
trainer = Trainer(
    model = model,
    args  =  args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)

C:\Users\danie\AppData\Local\Temp\ipykernel_7072\2426107787.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [111]:
torch.cuda.empty_cache()

In [112]:

trainer.train()


Epoch,Training Loss,Validation Loss,F1
1,No log,0.351909,0.882971
2,No log,0.264776,0.911324
3,No log,0.251558,0.942138
4,No log,0.255363,0.930566
5,No log,0.214275,0.947629
6,No log,0.396854,0.919613
7,No log,0.245725,0.935445
8,No log,0.222633,0.941550
9,No log,0.148182,0.966705
10,No log,0.153395,0.968528


TrainOutput(global_step=450, training_loss=0.05208275689019097, metrics={'train_runtime': 1142.6869, 'train_samples_per_second': 25.204, 'train_steps_per_second': 0.394, 'total_flos': 2.2565652942422016e+18, 'train_loss': 0.05208275689019097, 'epoch': 15.0})

In [133]:
sample = pd.read_csv(r'D:\data science\T1_hol\dataset\annotations\sample_submission.csv')
sample

,name,label
0,image_62214.jpg,NaN
1,image_91562.jpg,NaN
2,image_44104.jpg,NaN
3,image_79943.jpg,NaN
4,image_79847.jpg,NaN
...,...,...
595,image_58364.jpg,NaN
596,image_51853.jpg,NaN
597,image_44601.jpg,NaN
598,image_08599.jpg,NaN


In [134]:
sample['image_name'] = sample['name']

In [135]:
sample = sample.drop(['label','name'],axis = 1)

In [136]:
sample

,image_name
0,image_62214.jpg
1,image_91562.jpg
2,image_44104.jpg
3,image_79943.jpg
4,image_79847.jpg
...,...
595,image_58364.jpg
596,image_51853.jpg
597,image_44601.jpg
598,image_08599.jpg


In [137]:
sample['image_name'] =r'D:/data science/T1_hol/dataset/test/'+sample['image_name']

In [138]:
samp = Dataset.from_pandas(sample)

In [139]:
from datasets import Image, Dataset
testset = samp.cast_column("image_name", Image())

In [140]:
testset

Dataset({
    features: ['image_name'],
    num_rows: 600
})

In [141]:
import torch
from torchvision import transforms
from PIL import Image

def load_test(example_batch, target_size=(224, 224)):
    """
    Loads and preprocesses images from a test batch.

    Args:
        example_batch (dict): A batch of examples.
        target_size (tuple): The desired output size (height, width) of the images.

    Returns:
        dict: The processed batch with pixel values.
    """

    # Define the transformation pipeline
    transform = transforms.Compose([
        transforms.Resize(target_size),  # Resize to the target size
        transforms.ToTensor(),            # Convert to a PyTorch tensor
        transforms.Normalize(          # Normalize pixel values (adjust mean and std as needed)
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Process each image individually and collect pixel values as tensors
    di = []
    for image in example_batch["image_name"]:
        # Ensure image is a PIL Image object
        if isinstance(image, str):
            image = Image.open(image)

        # Convert to RGB if necessary
        if image.mode != "RGB":
            image = image.convert("RGB")

        # Apply transformations
        processed_image = transform(image)
        di.append(processed_image)

    # Stack the list of tensors into a single tensor with batch dimension
    example_batch["pixel_values"] = torch.stack(di)

    # Remove original column no longer needed
    del example_batch["image_name"]

    return example_batch

In [142]:
testset.set_transform(load_test)

In [143]:
testset

Dataset({
    features: ['image_name'],
    num_rows: 600
})

In [144]:
lst=[]
with torch.no_grad():
    for input in tqdm(testset):
        outputs = model(input['pixel_values'].to('cuda').reshape(1,3,224,224)).logits.argmax(-1)
        lst.append(int(outputs.cpu()))
lst[:5]

  0%|          | 0/600 [00:00<?, ?it/s]

[2, 2, 2, 2, 2]

In [145]:
sample['lables'] = lst 

In [146]:
sample

,image_name,lables
0,D:/data science/T1_hol/dataset/test/image_6221...,2
1,D:/data science/T1_hol/dataset/test/image_9156...,2
2,D:/data science/T1_hol/dataset/test/image_4410...,2
3,D:/data science/T1_hol/dataset/test/image_7994...,2
4,D:/data science/T1_hol/dataset/test/image_7984...,2
...,...,...
595,D:/data science/T1_hol/dataset/test/image_5836...,1
596,D:/data science/T1_hol/dataset/test/image_5185...,1
597,D:/data science/T1_hol/dataset/test/image_4460...,1
598,D:/data science/T1_hol/dataset/test/image_0859...,0


In [147]:
from collections import Counter
Counter(sample['lables'])

Counter({3: 125, 4: 121, 1: 120, 0: 120, 2: 114})

In [148]:
sample['lables'] = le1.inverse_transform(sample['lables'])

In [149]:
sample

,image_name,lables
0,D:/data science/T1_hol/dataset/test/image_6221...,rose
1,D:/data science/T1_hol/dataset/test/image_9156...,rose
2,D:/data science/T1_hol/dataset/test/image_4410...,rose
3,D:/data science/T1_hol/dataset/test/image_7994...,rose
4,D:/data science/T1_hol/dataset/test/image_7984...,rose
...,...,...
595,D:/data science/T1_hol/dataset/test/image_5836...,fern
596,D:/data science/T1_hol/dataset/test/image_5185...,fern
597,D:/data science/T1_hol/dataset/test/image_4460...,fern
598,D:/data science/T1_hol/dataset/test/image_0859...,cactus


In [150]:
sample['image_name'] = sample['image_name'].apply(lambda x: x[36:])

In [151]:
sample

,image_name,lables
0,image_62214.jpg,rose
1,image_91562.jpg,rose
2,image_44104.jpg,rose
3,image_79943.jpg,rose
4,image_79847.jpg,rose
...,...,...
595,image_58364.jpg,fern
596,image_51853.jpg,fern
597,image_44601.jpg,fern
598,image_08599.jpg,cactus


In [152]:
sample = sample.rename(columns={'image_name': 'name'})

In [153]:
sample

,name,lables
0,image_62214.jpg,rose
1,image_91562.jpg,rose
2,image_44104.jpg,rose
3,image_79943.jpg,rose
4,image_79847.jpg,rose
...,...,...
595,image_58364.jpg,fern
596,image_51853.jpg,fern
597,image_44601.jpg,fern
598,image_08599.jpg,cactus


In [154]:
sample.to_csv('hub_otv_l.csv',index =False)